### Optimizers


In [ ]:
# | default_exp optim
# | hide
import nbdev
from nbdev.showdoc import *

nbdev.nbdev_export()

In [ ]:
# | exporti

import numpy as np
from tidygrad.tensor import Tensor

In [ ]:
# | export

class Optimizer:
    def __init__(self, params):
        self.params = params

    def step(self):
        raise NotImplementedError

    def zero_grad(self):
        for p in self.params:
            p.grad.fill(0)

In [ ]:
# | export
class SGD(Optimizer):
    def __init__(self, params, lr=0.1, mom=0):
        super().__init__(params)
        self.lr = lr
        self.mom = mom
        self.moms = [np.zeros_like(p.data) for p in params]

    def step(self):
        for i, p in enumerate(self.params):
            if not np.isnan(p.grad).any() or np.isinf(p.grad).any():
                self.moms[i] = self.mom * self.moms[i] + (1 - self.mom) * p.grad
                p.data -= self.lr * self.moms[i]  # / (1 - self.mom)
            else:
                print("NaN gradient encountered, skipping step.")

In [ ]:
# | export
class Adam(Optimizer):
    def __init__(self, params, lr=0.001, beta1=0.9, beta2=0.999, eps=1e-8):
        super().__init__(params)
        self.lr = lr
        self.beta1 = beta1
        self.beta2 = beta2
        self.eps = eps
        self.ms = [np.zeros_like(p.data) for p in params]
        self.vs = [np.zeros_like(p.data) for p in params]
        self.t = 0

    def step(self):
        self.t += 1
        for i, p in enumerate(self.params):
            self.ms[i] = self.beta1 * self.ms[i] + (1 - self.beta1) * p.grad
            self.vs[i] = self.beta2 * self.vs[i] + (1 - self.beta2) * p.grad**2
            m_hat = self.ms[i] / (1 - self.beta1**self.t)
            v_hat = self.vs[i] / (1 - self.beta2**self.t)
            p.data -= self.lr * m_hat / (np.sqrt(v_hat) + self.eps)